In [7]:
!pip install transformers

In [2]:
from google.colab import files
import pandas as pd

# Upload the dataset
uploaded = files.upload()

# Load the dataset into a pandas dataframe
df = pd.read_csv(next(iter(uploaded)))

# Display the first few rows to understand the structure
df.head()


Saving LLM_dataset.csv to LLM_dataset.csv


,x_i,x_j,Label
0,he held her back and,"whispered urgently for her to stay calm, sensi...",GPT4o
1,none of her previous boyfriends had,had lasted this long.,GPT4o
2,"our clothes quickly disappeared , until we stood",naked in the moonlight.,GPT4o
3,but dagmar was not some mindless,follower; she had a plan.,GPT4o
4,"she paused , obviously",uncertain of what to say next.,GPT4o


In [4]:
# Check for missing values
df.isnull().sum()

# Combine `x_i` and `x_j` into a single text field if that’s part of your strategy
df['combined_text'] = df['x_i'] + ' ' + df['x_j']

# View a few rows of the combined text and labels
df[['combined_text', 'Label']].head()


,combined_text,Label
0,he held her back and whispered urgently for he...,GPT4o
1,none of her previous boyfriends had had lasted...,GPT4o
2,"our clothes quickly disappeared , until we sto...",GPT4o
3,but dagmar was not some mindless follower; she...,GPT4o
4,"she paused , obviously uncertain of what to sa...",GPT4o


In [6]:
from sklearn.model_selection import train_test_split

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(df['combined_text'], df['Label'], test_size=0.2, random_state=42)


In [10]:
from transformers import DistilBertTokenizer
print(X_train[:5])
# Load the DistilBERT tokenizer
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

X_train = [str(text) for text in X_train]
X_test = [str(text) for text in X_test]
# Tokenize the training and testing data
train_encodings = tokenizer(list(X_train), truncation=True, padding=True, max_length=512, return_tensors='pt')
test_encodings = tokenizer(list(X_test), truncation=True, padding=True, max_length=512, return_tensors='pt')


23311    i was n't sure if he was stunned he was simply...
23623    foley stood in her open doorway , a candle in ...
1020      but it ties in into the greater scheme of thi...
12645    she backed up hastily , and the scarf ed her t...
1533     i unwound myself from him and could myself fro...
Name: combined_text, dtype: object


In [11]:
import torch
from torch.utils.data import Dataset, DataLoader

class TextDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

# Convert labels to tensors
y_train = y_train.astype('category').cat.codes.tolist()  # Convert labels to integers
y_test = y_test.astype('category').cat.codes.tolist()

# Create PyTorch datasets
train_dataset = TextDataset(train_encodings, y_train)
test_dataset = TextDataset(test_encodings, y_test)

# Create DataLoader objects
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)


In [16]:
from transformers import DistilBertForSequenceClassification, AdamW

# Load the pre-trained DistilBERT model for classification
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=5)  # Assuming 3 labels

# Set up the optimizer
optimizer = AdamW(model.parameters(), lr=5e-5)

# Move the model to GPU if available
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

In [17]:
from tqdm import tqdm

# Training loop
model.train()
for epoch in range(3):  # Train for 3 epochs
    loop = tqdm(train_loader, leave=True)
    for batch in loop:
        # Move batch data to the same device as the model
        batch = {k: v.to(device) for k, v in batch.items()}

        # Forward pass
        outputs = model(**batch)
        loss = outputs.loss

        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Update progress bar
        loop.set_description(f'Epoch {epoch}')
        loop.set_postfix(loss=loss.item())


  0%|          | 0/1250 [00:00<?, ?it/s]<ipython-input-11-fa6cf741caec>:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
Epoch 2: 100%|██████████| 1250/1250 [02:52<00:00,  7.24it/s, loss=0.36]


In [23]:
from sklearn.metrics import accuracy_score, classification_report

# Put the model in evaluation mode
model.eval()

# Define the class names in the same order as the numerical labels
class_names = ['GPT2', 'GPT4o', 'GPT_NEO', 'Gemini', 'Reformer']

# Collect all predictions and labels
preds = []
true_labels = []

with torch.no_grad():
    for batch in test_loader:
        # Move batch to device
        batch = {k: v.to(device) for k, v in batch.items()}

        # Forward pass
        outputs = model(**batch)

        # Get predictions
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1).cpu().numpy()
        labels = batch['labels'].cpu().numpy()

        # Store predictions and true labels
        preds.extend(predictions)
        true_labels.extend(labels)

# Calculate accuracy and print classification report
accuracy = accuracy_score(true_labels, preds)
print(f'Accuracy: {accuracy}')
print(classification_report(true_labels, preds, target_names=class_names))


<ipython-input-11-fa6cf741caec>:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Accuracy: 0.6724
              precision    recall  f1-score   support

        GPT2       0.56      0.51      0.54      1000
       GPT4o       0.67      0.78      0.72      1021
     GPT_NEO       0.52      0.62      0.56       989
      Gemini       0.71      0.55      0.62      1005
    Reformer       0.95      0.90      0.92       985

    accuracy                           0.67      5000
   macro avg       0.68      0.67      0.67      5000
weighted avg       0.68      0.67      0.67      5000



In [27]:
import random

# X_test as lists
class_names = ['GPT2', 'GPT4o', 'GPT_NEO', 'Gemini', 'Reformer']

# Randomly choose 10 indices from the test set
random_indices = random.sample(range(len(X_test)), 10)

# Show the true label, predicted label, and whether they match
for idx in random_indices:
    # Get the text, true label, and predicted label for the selected index
    text = X_test[idx]  # Use X_test[idx]
    true_label = class_names[true_labels[idx]]  # Convert true label from int to class name
    predicted_label = class_names[preds[idx]]  # Convert predicted label from int to class name

    # Display the text, true label, predicted label, and whether they match
    print(f"Text: {text}")
    print(f"True Label: {true_label}")
    print(f"Predicted Label: {predicted_label}")
    print(f"Match: {true_label == predicted_label}")
    print('-' * 80)


Text: then she opened her seat and frightened of her left her head, went back and tears.
True Label: Reformer
Predicted Label: Reformer
Match: True
--------------------------------------------------------------------------------
Text: 'boromir was satisfied of that gradition was positively completely reflected by this time,
True Label: Reformer
Predicted Label: Reformer
Match: True
--------------------------------------------------------------------------------
Text: kelsey asked , not bothering to hide her curiosity.
True Label: GPT4o
Predicted Label: GPT4o
Match: True
--------------------------------------------------------------------------------
Text: he was about to press the zip-strip closed with a small, heavy-duty zip-strip.
True Label: GPT_NEO
Predicted Label: GPT_NEO
Match: True
--------------------------------------------------------------------------------
Text: not because he yelled or even tried to stop it, but because he was so angry.
True Label: GPT2
Predicted Label: GP

In [25]:
df['Label'] = df['Label'].astype('category')

# Print the mapping of categories to numbers
print(dict(enumerate(df['Label'].cat.categories)))


{0: 'GPT2', 1: 'GPT4o', 2: 'GPT_NEO', 3: 'Gemini', 4: 'Reformer'}
